# Read image stack

In [ ]:
import imageio
from pathlib import Path

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from tqdm.auto import tqdm

from astro3d import image_stack

## Settings

We again need to set the printer-specific settings and the directory of the slides:

In [ ]:
dpi_x = 600
dpi_y = 300
dz    = 27e-4 # 27 micron
# directory = 'slices_turbulent_box'
directory = 'slices_pluto_color2/'

## Read Images

Get number of files and file dimensions

In [ ]:
files = sorted(list(Path(directory).glob('*.png')))
im = imageio.imread(files[len(files)//2]).transpose(1, 0, 2)
_x = np.arange(im.shape[0])
_y = np.arange(im.shape[1])

dx = 2.54 / dpi_x
dy = 2.54 / dpi_y
dpi_z = 2.54 / dz

## Print out statistics

In [ ]:
cols = image_stack.check_colors(im)
plt.imshow([cols]).axes.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

Sum up the individual colors other than white. This can take a few minutes for large stacks.

In [ ]:
%time summed = image_stack.image_sum(files, colors=cols)#[:-1])

Check that every pixel is assigned one color and not more

In [ ]:
print(f'Material assignment is {int(summed.sum() != len(files) * np.prod(im.shape[:2])) * "NOT "}OK')

Show the distribution of materials

In [ ]:
nc = len(cols)# - 1

f, ax = plt.subplots(2, nc, figsize=(5 * nc, 5), gridspec_kw={'height_ratios': [1, 20]}, dpi=150)

for ic, col in enumerate(cols):#[:-1]):
    cc = ax[1, ic].imshow(summed[:, :, ic],
                          vmin=0,
                          vmax=5,#summed[:, :, ic].max(),
                          cmap=LinearSegmentedColormap.from_list('my', [[1,1,1], cols[ic] / 255]))
    ax[1, ic].set_aspect(0.5)
    f.colorbar(cc, cax=ax[0, ic], orientation='horizontal');

In [ ]:
print(f'{len(files)} files')
print(f'dimension = {im.shape[0] * dx:.2f} x {im.shape[1] * dy:.2f} x {len(files) * dz:.2f} cm')
print(f'nr of empty pixels: {(summed[:,:,:-1].sum(-1)==0).sum() / np.prod(summed.shape[:2]):.2%}')
print(f'most opaque pixel has {summed.max():n} filled pixels (={summed.max() / len(files):.2%} of all layers are filled)')

plt.imshow([cols]).axes.axis('off');
print('mean counts in non-transparent columns: ' + ', '.join([f'{np.mean(summed[:,:,i][summed[:,:,i]!=0]):.2g}' for i in range(len(cols))]))

In [ ]:
f, ax = plt.subplots(1, summed.shape[-1], figsize=(summed.shape[-1] * 4, 4), dpi=150)
for _ax, ic in zip(ax, range(summed.shape[-1])):
    _ax.imshow(summed[:, :, ic], origin='lower', vmin=0, vmax=1, cmap='gray_r')
    _ax.set_aspect(dpi_y / dpi_x)

In [ ]:
f, ax = plt.subplots()
_= ax.hist(summed[:,:, 0].ravel(), bins=np.linspace(0,20,21)-0.5);
ax.set_yscale('log')
ax.set_xlim(right=10)

## Plots

### Statistics plots
Show fully transparent pixels and the density distribution.

In [ ]:
f, axs = plt.subplots(3, 1, figsize=(4, 15), dpi=200)

summed_image = summed[:, :, :-1].sum(-1)

ax = axs[0]
cc=ax.imshow(summed_image>0, cmap='gray_r')
ax.set_aspect(dpi_y / dpi_x)
plt.colorbar(cc, ax=ax)
ax.set_title('white = all voxels transparent');

ax = axs[1]
ax.set_aspect(dpi_x / dpi_y)
i = ax.pcolormesh(_x, _y, summed_image.T, vmin=0, vmax=summed_image.max(), cmap='gray_r')
cb = plt.colorbar(i, ax=ax)
cb.set_label('# of opaque pixes along LOS')

ax = axs[-1]
counts, bins, patches = ax.hist(summed_image.ravel(), bins=np.arange(summed_image.max()))
ax.set_xlabel('number of filled voxels in column')
ax.set_xlabel('count');

### Compare data to 3d-printed result

In [ ]:
slice_width = .1 # in cm
n_slice = int(slice_width / dz)

In [ ]:
img = image_stack.image_sum(files[:n_slice])

In [ ]:
f, axs = plt.subplots(1, 3, dpi=200, figsize=(15,5))
axs[0].imshow(img.T / img.max(), cmap='gray_r')
axs[0].set_aspect(dpi_x / dpi_y)
axs[0].set_title(f'sum over first {slice_width:.2f} cm')

axs[1].imshow(imageio.imread('results/img1.jpg'))
axs[1].set_title('45º front illuminated')
axs[2].axis('off')

axs[2].imshow(imageio.imread('results/img2.jpg'))
axs[2].set_title('side illuminated')
axs[2].axis('off');

In [ ]:
from IPython.display import Video
Video('results/turbulent_box.mp4', width=500, html_attributes='autoplay loop')

# Top color

In [ ]:
im = imageio.imread(files[0])

dummyval = 300

im_set = dummyval * np.ones(im.shape, dtype=np.int32)
bg = np.array([255] * 3, dtype=np.int32)
empty = np.array([dummyval] * 3, dtype=np.int32)

for file in tqdm(files):
    im = imageio.imread(file)
    im_set = np.where((np.all(im_set==empty[None, None, :], -1) &  ~np.all(im==bg[None, None, :], -1))[:,:,None], im, im_set)

In [ ]:
f, ax = plt.subplots(dpi=300)
ax.imshow(np.minimum(np.maximum(im_set, 0), 255))
ax.set_aspect(dpi_x / dpi_y);